# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

## Queries 

In [11]:
## Artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

# Create table
query = "CREATE TABLE IF NOT EXISTS music_info1 "
query = query + "(artist text, song_title text, song_length float, session_id int, item_in_session int, PRIMARY KEY (session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

# Insert data
file = 'event_datafile_new.csv'

data = pd.read_csv(file)
for _, row in data.iterrows():
    query = "INSERT INTO music_info1 (artist, song_title, song_length, session_id, item_in_session)"
    query = query + " VALUES (%s, %s, %s, %s, %s)"
    try:
        session.execute(query, (row['artist'], row['song'], row['length'], row['sessionId'], row['itemInSession']))
    except Exception as e:
        print(e)

# Query data
query = "SELECT * FROM music_info1 WHERE session_id = 338 AND item_in_session = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song_title, row.song_length)
    


In [26]:
## Artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

# Create data
query = "CREATE TABLE IF NOT EXISTS music_info2 "
query = query + "(artist text, song_title text, item_in_session int, first_name text, last_name text, user_id int, session_id int, PRIMARY KEY (user_id, session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)


# Insert data
file = 'event_datafile_new.csv'

data = pd.read_csv(file)
for _, row in data.iterrows():
    query = "INSERT INTO music_info2 (artist, song_title, item_in_session, first_name, last_name, user_id, session_id)"
    query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
    try:
        session.execute(query, (row['artist'], row['song'], row['itemInSession'], row['firstName'], row['lastName'], row['userId'], row['sessionId'], ))
    except Exception as e:
        print(e)
        

# Query data
query = "SELECT * FROM music_info2 WHERE user_id = 10 AND session_id = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song_title, row.first_name, row.last_name, row.item_in_session)
    


Down To The Bone Keep On Keepin' On Sylvie Cruz 0
Three Drives Greece 2000 Sylvie Cruz 1
Sebastien Tellier Kilometer Sylvie Cruz 2
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz 3


In [29]:
## Query 3: Every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

# Create table
query = "CREATE TABLE IF NOT EXISTS music_info3 "
query = query + "(first_name text, last_name text, song_title text, PRIMARY KEY (song_title))"
try:
    session.execute(query)
except Exception as e:
    print(e)

                    
# Insert data
file = 'event_datafile_new.csv'

data = pd.read_csv(file)
for _, row in data.iterrows():
    query = "INSERT INTO music_info3 (first_name, last_name, song_title)"
    query = query + " VALUES (%s, %s, %s)"
    try:
        session.execute(query, (row['firstName'], row['lastName'], row['song']))
    except Exception as e:
        print(e)
        
        
# Query data
query = "SELECT * FROM music_info3 WHERE song_title = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.first_name, row.last_name)
                    

Jacqueline Lynch


### Drop the tables before closing out the sessions

In [30]:
try:
    session.execute("DROP TABLE music_info1")
except Exception as e:
    print(e)
    
try:
    session.execute("DROP TABLE music_info2")
except Exception as e:
    print(e)
    
try:
    session.execute("DROP TABLE music_info3")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [31]:
session.shutdown()
cluster.shutdown()